<a href="https://colab.research.google.com/github/birajsilwal/TweetSentiment/blob/master/TweetSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import re

# Filter tweets class

In [4]:

'''
this class filters (removes tweet tags, punctuations,
irrelevant words such as a, an, the) from the tweets
and returns array of clean filtered tweets
'''
class FilterTweets:

    def filterTweets(self, file_path):
      final_tweets = []
      tweets = []
      tweets_raw = open(file_path)
      for tweet in tweets_raw:
          tweet = tweet.lower()
          tweet = re.sub(":|;|\(|\)|!|\'s|\"|\.|n\'t|,|\?|i\'m|i\'ve", "", tweet)
          tweet = re.sub(" an | the | is | of | a | i | was | and |"
                          " on | in | off | all | it | me | you | to |"
                          " into | we | your | that | they | can | could |"
                          " should | do | does | for | my | at | so |"
                          " if | has | have | had | from | such | are |"
                          " not | this | now | but | go | day |"
                          "-|_| up | down | these | today | lol |"
                          " lmao | af | get | got | here | there | who |"
                          " what | am | no | why | with | us | our | bro |"
                          " too | then | ur | zero | ah | see | saw ", " ", tweet)
          tweets.append(tweet)


      # cleaning up twice because of space issue
      tweets1 = []
      for tweet in tweets:
          tweet = tweet.lower()
          tweet = re.sub(" an | the | is | of | a | i | was | and |"
                          " on | in | off | all | it | me | you | to |"
                          " into | we | your | that | they | can | could |"
                          " should | do | does | for | my | at | so |"
                          " if | has | have | had | from | such | are |"
                          " not | this | now | but | go | day |"
                          "-|_| up | down | these | today | lol |"
                          " lmao | af | get | got | here | there | who |"
                          " what | am | no | why | with | us | our | bro |"
                          " too | then | ur | zero | ah | see | saw ", " ", tweet)
          tweets1.append(tweet)


      '''
      if there contains a substring starting with http or @,
      remove that substring from the tweet
      '''
      for str in tweets1:
          index = 0
          string_arr = str.split()
          for string in string_arr:
              if 'http' in string or '@' in string:
                  string_arr[index] = string.replace(string, '')
              index += 1

          tweet = ' '
          tweet = tweet.join(string_arr)
          final_tweets.append(tweet)
          # print(tweet)
      return final_tweets

# Filtered datasets

In [5]:
filter_tweets = FilterTweets()

tweets_training_filtered = filter_tweets.filterTweets('/content/drive/MyDrive/Colab Notebooks/Tweets/noCR_train.txt')
print(len(tweets_training_filtered))

tweets_testing_filtered = filter_tweets.filterTweets('/content/drive/MyDrive/Colab Notebooks/Tweets/noCR_test.txt')
print(len(tweets_testing_filtered))

10271
1145


# Tweet class

In [6]:
class Tweet:
  def __init__(self, tweet, zero_or_one):
      self.tweet = tweet
      self.zero_or_one = zero_or_one
      self.sentiment = self.get_sentiment()

  
  def get_sentiment(self):
    if self.zero_or_one == '0':
      return "NEGATIVE"
    elif self.zero_or_one == '1':
      return "POSITIVE" 


In [7]:
# creating tweet objects for each tweet
tweets_training = []
tweets_testing = []

for tweet in tweets_training_filtered:
  string_arr = tweet.split()
  zero_or_one = string_arr.pop(0)

  tweet1 = ' '
  tweet1 = tweet1.join(string_arr)
  tweets_training.append(Tweet(tweet1, zero_or_one))


for tweet in tweets_testing_filtered:
  string_arr = tweet.split()
  zero_or_one = string_arr.pop(0)
  
  tweet1 = ' '
  tweet1 = tweet1.join(string_arr)
  tweets_testing.append(Tweet(tweet1, zero_or_one))

In [8]:
# x and y values for both training and testing dataset
train_x = [x.tweet for x in tweets_training]
train_y = [x.sentiment for x in tweets_training]

test_x = [x.tweet for x in tweets_testing]
test_y = [x.sentiment for x in tweets_testing]

In [9]:
def update_dict():
  dict_tweet = {}
  val = 1

  for tweet in train_x:
      tweet_arr = tweet.split()

      for x in range(len(tweet_arr)):
          key = tweet_arr[x]
          '''
          see if the key already exists in the dictionary
          if yes, increase the value by 1 otherwise
          new key and val.
          '''
          if key in dict_tweet:
              update_val = dict_tweet[key] + 1
              update_dict = {key: update_val}
              dict_tweet.update(update_dict)
          else:
              dict_tweet[key] = val

# Vectorization

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x) 

# Naive Bayes

In [11]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(train_x_vectors.toarray(), train_y)

clf.predict(test_x_vectors[99].toarray())

array(['NEGATIVE'], dtype='<U8')

In [12]:
print(clf.score(test_x_vectors.toarray(), test_y))

0.6541484716157205


In [13]:
test_set = ["Thanks for your contribution and kind comment", 'horrible waste of time']
new_test = vectorizer.transform(test_set)

clf.predict(new_test.toarray()) 

array(['POSITIVE', 'NEGATIVE'], dtype='<U8')